In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import requests
import re
import pandas as pd
import numpy as np
import os
from selenium.webdriver.common.keys import Keys

In [ ]:
name=['맥북 에어 2020년형 M1 256G 스페이스 그레이 MGN63KH/A']
category=['별점']

#맥북 에어 2020년형
ns_address="https://search.shopping.naver.com/catalog/26101847522?NaPm=ct%3Dkxdzxb6o%7Cci%3Dcf506a4869c6e53545f97dcd7ebbf2fbb0eed6dc%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3Dc54fe62bc7e4c7d4c5ce71a4dd873f1ffee467b2"
#xpath
shoppingmall_review="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[2]/div/div[2]/ul/li[4]/a/strong"
category1="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/ul/li[2]/a" #5점
category2="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/ul/li[3]/a" #4점
category3="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/ul/li[4]/a" #3점
category4="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/ul/li[5]/a" #2점
category5="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/ul/li[6]/a" #1점

In [9]:
header = {'User-Agent': ''}
d = webdriver.Chrome('C:/Users/mewtw/Downloads/project06/chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address)
req = requests.get(ns_address,verify=False)
html = req.text 
soup = BeautifulSoup(html, "html.parser")
sleep(2)

#쇼핑몰 리뷰 보기
#d.find_element_by_xpath(shoppingmall_review).click()
#sleep(2)

element=d.find_element_by_xpath(shoppingmall_review)
d.execute_script("arguments[0].click();", element)
sleep(2)

<ipython-input-9-5026750bc7d0>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d = webdriver.Chrome('C:/Users/mewtw/Downloads/project06/chromedriver.exe') # webdriver = chrome
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search.shopping.naver.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
<ipython-input-9-5026750bc7d0>:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element=d.find_element_by_xpath(shoppingmall_review)


In [5]:
#함수 선언
def add_dataframe(name,category,reviews,stars,cnt):  #데이터 프레임에 저장
    #데이터 프레임생성
    df1=pd.DataFrame(columns=['type','category','review','star']) #keyword #별점 #리뷰 text #5-1리뷰
    n=1
    if (cnt>0):
        for i in range(0,cnt-1):
            df1.loc[n]=[name,category,reviews[i],stars[i]] #해당 행에 저장
            i+=1
            n+=1
    else:
        df1.loc[n]=[name,category,'null','null']
        n+=1    
    return df1

In [11]:
def save():
    if not os.path.exists('output.csv'):
        df1.to_csv('output.csv', encoding='utf-8-sig', mode='w')
    else:
        df1.to_csv('output.csv',encoding='utf-8-sig', mode='a',header=False)
    

In [12]:
# 5점 리뷰 가져오기
d.find_element_by_xpath(category1).click() #스크롤 건드리면 안됨
name_=name[0]
category_=category[0]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1


while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li[1]/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]/p').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]/p')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            page +=1
            next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click() 
            
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            page+=1
            if page%10==0: 
                next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page%10+2)+']').click()
            
        except: break 
            
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()

# 4점 리뷰 가져오기
up="/html/body/div/div/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category2).click() #스크롤 건드리면 안됨
name_=name[0]
category_=category[0]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1


while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li[1]/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]/p').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]/p')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            page +=1
            next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click() 
            
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            page+=1
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page%10+2)+']').click()
            
        except: break 
            
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()

# 3점 리뷰 가져오기
up="/html/body/div/div/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category3).click() #스크롤 건드리면 안됨
name_=name[0]
category_=category[0]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1


while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li[1]/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]/p').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]/p')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            page +=1
            next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click() 
            
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            page+=1
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page%10+2)+']').click()
            
        except: break 
            
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()

# 2점 리뷰 가져오기
up="/html/body/div/div/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category4).click() #스크롤 건드리면 안됨
name_=name[0]
category_=category[0]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1


while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li[1]/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]/p').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]/p')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            page +=1
            next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click() 
            
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            page+=1
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page%10+2)+']').click()
            
        except: break 
            
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()

# 1점 리뷰 가져오기
up="/html/body/div/div/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category5).click() #스크롤 건드리면 안됨
name_=name[0]
category_=category[0]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1


while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li[1]/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]/p').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]/p')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            page +=1
            next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click() 
            
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            page+=1
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page%10+2)+']').click()
            
        except: break 
            
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()


<ipython-input-12-51bffe96a642>:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  d.find_element_by_xpath(category1).click() #스크롤 건드리면 안됨


페이지 1 



<ipython-input-12-51bffe96a642>:17: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li[1]/div[1]/span[1]').text
<ipython-input-12-51bffe96a642>:19: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]/p').text
<ipython-input-12-51bffe96a642>:22: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]/p')


1 맥북 말해뭐합니까,,, 너무 예쁘고요,, 영롱하고요,, 그냥 보기만해도 힐링입니다 음질은 또 얼마나 좋은지 음악 들을때마다 고막이 힐링이에요 다만 저는 게임을 즐겨하는편이 아님에도 가끔 카트라이더가 하고싶은데 게임이 안된다는 단점만 빼면 완벽합니다! 평점5 

2 딸아이가 평소 넘 갖고 싶어 하는건데..세일한다고 이 기회 놓치기 싫다고 하더군요..
모아놓은 용돈이 부족하여 제가 조금 빌려줬네요..하루만에 배송된 맥북을 보고 어찌나 좋아라 하는지..다른 노트북이 있지만, 이게 더 맘에 든다는군요..ㅡ.ㅡ;;
잘 모르는 저로써는 차이가 먼가 싶지만,
좋아하는 아이얼굴을 보니 사길 잘한것 같네요 평점5 

3 선물용으로 구매했는데, 선물주인이 아주 행복해하네요:)
배송도 안전하고 신속하게 진행되었고, 제조일이 구매일과 같은 달 상품이라 왠지모를 안심이 되더라구요!
선물증정식 후 언박싱 했을 때 맥북에어의 영롱함,,
특히 스페이스그레이만의 고급짐에 감탄했어요.
애플이 애플 했네요..!
사용하시는 분이 평소에 영상을 많이 보는데, 화질&음질이 너무 좋대요.
키보드의 촉감도 가볍고, 무엇보다 오래 사용해도 발열이 심하지 않은 점이 제일 장점인 것 같아요.
애플제품 치고 성능대비 가격이 합리적인 편인데, SSG 현대카드 혜택이 많길래 신규발급 후 기본할인에 청구할인까지 받으니 감동적인 금액에 구입할 수 있었어요.
한가지 아쉬운 점은, 선물용으로 구매하시는 분들이 있는 것 같은데 선물용 종이가방이 옵션으로 있으면 좋을 것 같아요!
좋은제품 구매할 수 있게 해 주셔서 감사합니다.
소중한 사람에게 간지와 삶의 질 향상을 동시에 전해주어 뿌듯함과 희열을 느낍니다!
앞으로 나올 신제품들도 너무너무 기대되네요~ 평점5 

4 실버와 스페이스그레이 중 고민고민하다 결국 스그로 구매했습니다!
생각지도 못했는데 배송도 정말 빨라서 좋았어요.
그간 윈도 기반 랩탑만 쓰다가 처음 맥북을 구매하게 되었는데 좋은 기회에 잘 구입한 것 같아요.
아직 사용법을 익혀나가느라 솔직히 구체적으로 뭐가

<ipython-input-12-51bffe96a642>:34: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click()


...

<ipython-input-12-51bffe96a642>:185: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click()
<ipython-input-12-51bffe96a642>:202: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
<ipython-input-12-51bffe96a642>:203: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  d.find_element_by_xpath(category5).click() #스크롤 건드리면 안됨


페이지 1 



<ipython-input-12-51bffe96a642>:218: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li[1]/div[1]/span[1]').text
<ipython-input-12-51bffe96a642>:220: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]/p').text
<ipython-input-12-51bffe96a642>:223: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]/p')


1 10.3 주문 10.8배송배송은 얼마든지 느려질수 있다고 생각하나 공지도 없었으며, 택배는 발송했으니 배송관련해서는 택배사에 문의하라고 하지를 않나, 질문에 대한 무성의한 답변 등 간만에 멋진(?) 판매자에게 구매했음.반품하고 싶었으나 배송비 부담하라고 할 것이 뻔해보이고, 구매를 해야만 이러한 평을 남길 수 있어서 구매결정함.(진상고객으로 보여질 수 있으나, 단지 배송관련해서 답답했을 뿐입니다.) 평점1 

2 제품은 좋아요... 배송도 빨랐구요, 근데 "공식 애플 스토어" 라고 달아놓은게 맞긴한건가요? ㅠ 당연히 공식 리셀러라고 생각하고 제품 샀는데 애플에서는 여기가 공식 리셀러가 아니라고 하고... 공식리셀러 인증이 안되어서 회사용 기기등록도 못했어요 ㅠ 공식 리셀러시면 리셀러 ID를 알려주세요 ㅠ ABM등록해야해요 ㅠㅠㅠ 평점1 

3 저 화면에서 렉이걸려서 암것도안되고
강제종료조차 안돼고 별루네요... 평점1 

4 대구에서 서울까지 오는데 박스포장을 못하면 뽁뽁이라도 해서 보냈어야죠, 노트북만 달랑 보냅니까? 포장좀 신경쓰세요! 평점1 

5 주문한지 한달 넘었고 전화도 안 받네요. 더 걸릴 것 같으면 취소라도 하고 싶은데. 평점1 



<ipython-input-12-51bffe96a642>:235: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click()


In [13]:
dataset = pd.read_csv('output.csv', index_col=0)  #csv에서 가져오기
print(dataset)

                                       type category  \
1   맥북 에어 2020년형 M1 256G 스페이스 그레이 MGN63KH/A       별점   
2   맥북 에어 2020년형 M1 256G 스페이스 그레이 MGN63KH/A       별점   
3   맥북 에어 2020년형 M1 256G 스페이스 그레이 MGN63KH/A       별점   
4   맥북 에어 2020년형 M1 256G 스페이스 그레이 MGN63KH/A       별점   
5   맥북 에어 2020년형 M1 256G 스페이스 그레이 MGN63KH/A       별점   
..                                      ...      ...   
1   맥북 에어 2020년형 M1 256G 스페이스 그레이 MGN63KH/A       별점   
2   맥북 에어 2020년형 M1 256G 스페이스 그레이 MGN63KH/A       별점   
3   맥북 에어 2020년형 M1 256G 스페이스 그레이 MGN63KH/A       별점   
4   맥북 에어 2020년형 M1 256G 스페이스 그레이 MGN63KH/A       별점   
5   맥북 에어 2020년형 M1 256G 스페이스 그레이 MGN63KH/A       별점   

                                               review star  
1   맥북 말해뭐합니까,,, 너무 예쁘고요,, 영롱하고요,, 그냥 보기만해도 힐링입니다 ...  평점5  
2   딸아이가 평소 넘 갖고 싶어 하는건데..세일한다고 이 기회 놓치기 싫다고 하더군요....  평점5  
3   선물용으로 구매했는데, 선물주인이 아주 행복해하네요:)\n배송도 안전하고 신속하게 ...  평점5  
4   실버와 스페이스그레이 중 고민고민하다 결국 스그로 구매했습니다!\n생각지도 못했는데...  평점5  
5   저렴하게 좋은 제품 잘구한거 같습

In [14]:
dataset.head()

,type,category,review,star
1,맥북 에어 2020년형 M1 256G 스페이스 그레이 MGN63KH/A,별점,"맥북 말해뭐합니까,,, 너무 예쁘고요,, 영롱하고요,, 그냥 보기만해도 힐링입니다 ...",평점5
2,맥북 에어 2020년형 M1 256G 스페이스 그레이 MGN63KH/A,별점,딸아이가 평소 넘 갖고 싶어 하는건데..세일한다고 이 기회 놓치기 싫다고 하더군요....,평점5
3,맥북 에어 2020년형 M1 256G 스페이스 그레이 MGN63KH/A,별점,"선물용으로 구매했는데, 선물주인이 아주 행복해하네요:)\n배송도 안전하고 신속하게 ...",평점5
4,맥북 에어 2020년형 M1 256G 스페이스 그레이 MGN63KH/A,별점,실버와 스페이스그레이 중 고민고민하다 결국 스그로 구매했습니다!\n생각지도 못했는데...,평점5
5,맥북 에어 2020년형 M1 256G 스페이스 그레이 MGN63KH/A,별점,저렴하게 좋은 제품 잘구한거 같습니다 ㅎㅎ 이쁘고\n좋네요 ^^ 배송도 매우 빠르고...,평점5
